In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import scipy.linalg as linalg
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn as skl
import math
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from typing import List, Any 
from collections import Counter

ModuleNotFoundError: No module named 'pandas'

In [ ]:
video_games = pd.read_csv("data/Jeux_Videos.csv", sep=",")

In [ ]:
class Variable:
    genres = "genres" # qualitative nominal 
    overall_review = "overall_review" # qualitative ordinal
    overall_review_percentage = "overall_review_%" # quantitative continue 
    awards = "awards" # quantitative discrete
    categories = "categories"  # qualitative nominal
    developer = "developer"  # qualitative nominal
    publisher = "publisher"  # qualitative nominal
    discounted_price = "discounted_price"  # quantitative continue
    dlc_available = "dlc_available"  # quantitative discret
    age_rating = "age_rating"  # qualitative nominal

    original_price = "original_price"
    discount_percentage = "discount_percentage"
    
    overall_review_order = [
        "Overwhelmingly Negative", "Very Negative", "Negative", "Mostly Negative",
        "Mixed", "Mostly Positive", "Positive", "Very Positive", "Overwhelmingly Positive"
    ]


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Si besoin, on ordonne les notes dans le bon ordre
overall_order = [
    "Overwhelmingly Negative", "Very Negative", "Negative", "Mostly Negative",
    "Mixed", "Mostly Positive", "Positive", "Very Positive", "Overwhelmingly Positive"
]

# Charger les données (df = ton DataFrame déjà nettoyé)
df = df[df['genres'].notnull() & df['overall_review'].notnull()]
df['overall_review'] = pd.Categorical(df['overall_review'], categories=overall_order, ordered=True)

# Extraire le genre principal si plusieurs genres sont listés
df['main_genre'] = df['genres'].str.split(',').str[0]

# Graphique en pourcentage d’évaluations par genre principal
genre_review_ct = pd.crosstab(df['main_genre'], df['overall_review'], normalize='index') * 100
genre_review_ct = genre_review_ct[overall_order]  # garantir le bon ordre

# Affichage du graphique
plt.figure(figsize=(12, 6))
genre_review_ct.plot(kind='bar', stacked=True, colormap='viridis')
plt.title("Répartition des évaluations par genre principal")
plt.ylabel("Pourcentage des évaluations (%)")
plt.xlabel("Genre principal")
plt.legend(title="Évaluation globale", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Ordre des reviews
review_order = [
    "Overwhelmingly Negative", "Very Negative", "Negative", "Mostly Negative",
    "Mixed", "Mostly Positive", "Positive", "Very Positive", "Overwhelmingly Positive"
]

# S'assurer que la variable est bien ordonnée
df = df[df["overall_review"].isin(review_order)].copy()
df["overall_review"] = pd.Categorical(df["overall_review"], categories=review_order, ordered=True)

# Nettoyage simple : enlever les prix négatifs ou nuls
df = df[df["discounted_price"] > 0]

# Statistiques descriptives
group_stats = df.groupby("overall_review")["discounted_price"].describe()
print(group_stats)

# Visualisation
plt.figure(figsize=(14, 7))
sns.boxplot(x="overall_review", y="discounted_price", data=df, order=review_order)
plt.xticks(rotation=45)
plt.xlabel("Évaluation globale du jeu")
plt.ylabel("Prix après réduction (€)")
plt.title("Prix après réduction en fonction des évaluations des joueurs")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Chargement des données
video_games = pd.read_csv("data/Jeux_Videos.csv", sep=",")

# Ordre pour la variable ordinale
review_order = [
    "Overwhelmingly Negative", "Very Negative", "Negative", "Mostly Negative",
    "Mixed", "Mostly Positive", "Positive", "Very Positive", "Overwhelmingly Positive"
]

# Nettoyage : suppression des valeurs manquantes
df = video_games[[Variable.genres, Variable.overall_review]].dropna()

# Conversion en catégorie ordonnée
df[Variable.overall_review] = pd.Categorical(
    df[Variable.overall_review],
    categories=Variable.overall_review_order,
    ordered=True
)

# Optionnel : ne garder qu’un seul genre (si plusieurs genres séparés par virgule)
df[Variable.genres] = df[Variable.genres].str.split(",").str[0].str.strip()

# Crosstab : pourcentages par ligne (par niveau de review)
ct = pd.crosstab(df[Variable.overall_review], df[Variable.genres], normalize='index') * 100

# Affichage du tableau croisé
display(ct.round(1))

# Visualisation heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(ct, annot=False, cmap='YlGnBu', cbar_kws={'label': '%'})
plt.title('Répartition des genres selon la note globale (overall_review)')
plt.xlabel('Genre')
plt.ylabel('Avis global')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Nettoyage des données
df = video_games[[Variable.overall_review, Variable.original_price]].dropna()

# Conversion de la variable ordinale
df[Variable.overall_review] = pd.Categorical(
    df[Variable.overall_review],
    categories=Variable.overall_review_order,
    ordered=True
)

# Boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(x=Variable.overall_review, y=Variable.original_price, data=df)
plt.xticks(rotation=45, ha='right')
plt.title("Répartition du prix original selon l'avis global")
plt.xlabel("Avis global")
plt.ylabel("Prix original (€)")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

import re

def clean_price(value):
    if isinstance(value, str):
        # Supprime tout sauf les chiffres et le point
        value = re.sub(r"[^\d.]", "", value)
    try:
        return float(value)
    except:
        return None

def clean_percentage(value):
    if isinstance(value, str):
        value = value.replace("%", "").strip()
    try:
        return float(value)
    except:
        return None

# Applique le nettoyage
df = video_games[[Variable.original_price, Variable.discount_percentage]].copy()
df[Variable.original_price] = df[Variable.original_price].apply(clean_price)
df[Variable.discount_percentage] = df[Variable.discount_percentage].apply(clean_percentage)

# Supprime les NaN
df = df.dropna()

# Filtre des valeurs aberrantes
df = df[(df[Variable.original_price] > 0) & (df[Variable.original_price] < 500)]


# Vérification de la taille
if df.empty:
    print("❌ Aucune donnée valide après le nettoyage. Vérifie les valeurs ou les filtres.")
else:
    # Graphique de régression
    plt.figure(figsize=(10, 6))
    sns.regplot(
        x=Variable.original_price,
        y=Variable.discount_percentage,
        data=df,
        scatter_kws={'alpha': 0.5},
        line_kws={'color': 'red'}
    )
    plt.title("Régression linéaire : réduction (%) vs prix original")
    plt.xlabel("Prix original (€)")
    plt.ylabel("Pourcentage de réduction (%)")
    plt.tight_layout()
    plt.show()

    # Régression linéaire avec sklearn
    X = df[[Variable.original_price]]
    y = df[Variable.discount_percentage]
    model = LinearRegression().fit(X, y)
    print(f"📈 Régression linéaire : y = {model.coef_[0]:.2f}x + {model.intercept_:.2f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Charger les données
df = video_games

# Étape 1 : Nettoyage des données
# Convertir original_price et discounted_price en numérique
def clean_price(price):
    if pd.isna(price) or price == 'Free':
        return 0.0
    price = str(price).replace('₹', '').replace(',', '').strip()
    try:
        return float(price)
    except:
        return np.nan

df['original_price'] = df['original_price'].apply(clean_price)
df['discounted_price'] = df['discounted_price'].apply(clean_price)

# Convertir discount_percentage en numérique
def clean_discount(discount):
    if pd.isna(discount) or discount == '':
        return 0.0
    discount = str(discount).replace('%', '').strip()
    try:
        return float(discount)
    except:
        return 0.0

df['discount_percentage'] = df['discount_percentage'].apply(clean_discount)

# Remplacer les NaN dans overall_review_% et overall_review_count
df['overall_review_%'].fillna(df['overall_review_%'].median(), inplace=True)
df['overall_review_count'].fillna(0, inplace=True)

# Variables explicatives quantitatives
quant_vars = ['original_price', 'discount_percentage', 'overall_review_count', 
              'awards', 'dlc_available']
# Variable cible
target = 'overall_review_%'

# Étape 2 : Gestion des valeurs manquantes pour les variables quantitatives
for var in quant_vars:
    df[var].fillna(df[var].median(), inplace=True)

# Étape 3 : Régression linéaire multiple
X = df[quant_vars]
y = df[target]

# Standardisation des variables explicatives
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Modèle de régression
reg = LinearRegression()
reg.fit(X_scaled, y)

# Coefficients et intercept
print("Coefficients de la régression linéaire multiple :")
for var, coef in zip(quant_vars, reg.coef_):
    print(f"{var}: {coef:.4f}")
print(f"Intercept: {reg.intercept_:.4f}")

# Score R²
print(f"R²: {reg.score(X_scaled, y):.4f}")

# Étape 4 : Analyse en Composantes Principales (ACP)
# Sélection des variables quantitatives pour l'ACP
acp_vars = quant_vars + [target]
X_acp = df[acp_vars].dropna()

# Standardisation
X_acp_scaled = scaler.fit_transform(X_acp)

# ACP
pca = PCA()
X_pca = pca.fit_transform(X_acp_scaled)

# Variance expliquée
explained_variance_ratio = pca.explained_variance_ratio_
print("\nVariance expliquée par composante :")
for i, var in enumerate(explained_variance_ratio):
    print(f"Composante {i+1}: {var:.4f} ({var*100:.2f}%)")

# Visualisation de l'ACP
plt.figure(figsize=(8, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5)
plt.xlabel(f'Composante 1 ({explained_variance_ratio[0]*100:.2f}%)')
plt.ylabel(f'Composante 2 ({explained_variance_ratio[1]*100:.2f}%)')
plt.title('Projection des jeux sur les deux premières composantes principales')
plt.grid(True)

# Cercle des corrélations
plt.figure(figsize=(8, 6))
for i, var in enumerate(acp_vars):
    plt.arrow(0, 0, pca.components_[0, i], pca.components_[1, i], 
              color='r', alpha=0.5)
    plt.text(pca.components_[0, i]*1.15, pca.components_[1, i]*1.15, 
             var, color='r', ha='center', va='center')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.xlabel('Composante 1')
plt.ylabel('Composante 2')
plt.title('Cercle des corrélations (ACP)')
plt.grid(True)
circle = plt.Circle((0, 0), 1, color='b', fill=False)
plt.gca().add_artist(circle)
plt.show()

# Étape 5 : Analyse des corrélations
corr_matrix = df[acp_vars].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Matrice de corrélation des variables quantitatives')
plt.show()

In [ ]:
from docx import Document

# Créer un document Word
doc = Document()
doc.add_heading('Tableau croisé : Avis global vs Genre', level=1)

# Récupérer le tableau croisé
table_data = ct.round(1).reset_index()  # remettre 'overall_review' en colonne

# Créer le tableau dans Word
table = doc.add_table(rows=1, cols=len(table_data.columns))
table.style = 'Table Grid'

# Ajouter les en-têtes
hdr_cells = table.rows[0].cells
for i, col in enumerate(table_data.columns):
    hdr_cells[i].text = str(col)

# Ajouter les lignes
for _, row in table_data.iterrows():
    row_cells = table.add_row().cells
    for i, item in enumerate(row):
        row_cells[i].text = str(item)

# Sauvegarder
doc.save('tableau_croise.docx')
print("Fichier Word généré : tableau_croise.docx")


In [ ]:
def statistical_analysis_quantitative(series: pd.Series, cutted: bool = False, include_graph: bool = False) -> None:  
    print(f"\n Taille de la population : {series.count()} observations")  
    print(f" Minimum : {series.min()}")  
    print(f" Maximum : {series.max()}")  

    if cutted:
        # Calcul du pas
        num_bins = math.floor(math.sqrt(series.count()))
        
        # Découpage en classes
        cutted_series = pd.cut(series, bins=num_bins, precision=2)

        # Effectif par classe
        bin_counts = series.groupby(cutted_series).count()
        bin_cumulative_counts = bin_counts.cumsum()
        
        # Fréquences
        bin_frequencies = bin_counts / bin_counts.sum()
        bin_cumulative_frequencies = bin_frequencies.cumsum()

        # Médianes et modes
        medians = series.groupby(cutted_series).median().reindex(bin_counts.index)
        modes = series.groupby(cutted_series).apply(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan).reindex(bin_counts.index)

        # Création du tableau avec alignement des longueurs
        df_bins = pd.DataFrame({
            "Classe": bin_counts.index.astype(str),
            "Effectif": bin_counts.values,
            "Effectif cumulé": bin_cumulative_counts.values,
            "Fréquence": bin_frequencies.values,
            "Fréquence cumulée": bin_cumulative_frequencies.values,
            "Médiane": medians.values,
            "Mode": modes.values
        })

        print("\n Analyse par classes :")
        print(df_bins.to_string(index=False, formatters={
            "Fréquence": "{:.2%}".format, 
            "Fréquence cumulée": "{:.2%}".format
        }))

    else:
        # Analyse par valeur unique
        counts = series.value_counts().sort_index()
        cumulative_counts = counts.cumsum()
        frequencies = counts / series.count()
        cumulative_frequencies = frequencies.cumsum()

        df_stats = pd.DataFrame({
            "Valeur": counts.index,
            "Effectif": counts.values,
            "Effectif cumulé": cumulative_counts.values,
            "Fréquence": frequencies.values,
            "Fréquence cumulée": cumulative_frequencies.values
        })

        print("\n Analyse des valeurs uniques :")
        print(df_stats.to_string(index=False, formatters={
            "Fréquence": "{:.2%}".format, 
            "Fréquence cumulée": "{:.2%}".format
        }))

        print("\n Statistiques principales :")
        print(f"- Moyenne : {series.mean():.3f}")
        print(f"- Médiane : {series.median():.3f}")
        print(f"- Mode(s) : {', '.join(map(str, series.mode().tolist()))}")
        print(f"- 25% (Q1) : {series.quantile(0.25):.3f}")
        print(f"- 50% (Q2 - Médiane) : {series.quantile(0.5):.3f}")
        print(f"- 75% (Q3) : {series.quantile(0.75):.3f}")
        print(f"- Étendue : {series.max() - series.min():.3f}")
        print(f"- Écart interquartile (Q3 - Q1) : {series.quantile(0.75) - series.quantile(0.25):.3f}")
        print(f"- Écart-type : {series.std():.3f}")
        print(f"- Variance : {series.var():.3f}")

    # Affichage des graphiques
    if include_graph:
        fig, ax = plt.subplots(1, 3, figsize=(18, 5))

        # Histogramme
        ax[0].hist(series.dropna(), bins=math.floor(math.sqrt(series.count())), color="skyblue", alpha=0.7, edgecolor="black")
        ax[0].set_xlabel("Valeurs")
        ax[0].set_ylabel("Effectif")
        ax[0].set_title("Histogramme des valeurs")

        # CDF (Courbe de répartition empirique)
        sorted_data = np.sort(series.dropna())
        cdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
        ax[1].plot(sorted_data, cdf, marker="o", linestyle="-", color="orange")
        ax[1].set_xlabel("Valeurs")
        ax[1].set_ylabel("Probabilité cumulée")
        ax[1].set_title("Fonction de répartition empirique (CDF)")

        # Boxplot
        ax[2].boxplot(series.dropna(), vert=False, patch_artist=True, boxprops=dict(facecolor="lightblue"))
        ax[2].set_title("Boxplot de la variable")

        plt.tight_layout()
        plt.show()


In [ ]:
def statistical_analysis_qualitative_ordinal(series: pd.Series, include_graph: bool = False, custom_order: List[Any] = []) -> None:
    print(f"\n Taille de la population : {series.count()} observations")  # Nombre total d'observations
    
    # Vérifier si un ordre personnalisé est fourni
    if custom_order:
        value_counts = series.value_counts()
        counts = value_counts.reindex(custom_order).dropna()
    else:
        counts = series.value_counts().sort_index()  # Tri automatique si pas d'ordre personnalisé

    # Calcul des effectifs cumulés
    cumulative_counts = counts.cumsum()

    # Calcul des fréquences
    frequencies = counts / series.count()
    cumulative_frequencies = frequencies.cumsum()

    # Détermination du mode (valeur la plus fréquente)
    max_effectif = counts.max()
    mode_values = counts[counts == max_effectif].index.tolist()

    # Détermination de la médiane (valeur centrale)
    cumulative_counts_half = cumulative_counts[cumulative_counts >= series.count() / 2]
    mediane = cumulative_counts_half.index[0] if not cumulative_counts_half.empty else "Indéterminée"

    # Création du DataFrame pour affichage
    df_stats = pd.DataFrame({
        "Valeur": counts.index,
        "Effectif": counts.values,
        "Effectif cumulé": cumulative_counts.values,
        "Fréquence": frequencies.values,
        "Fréquence cumulée": cumulative_frequencies.values
    })

    # Affichage du tableau formaté
    print("\n Analyse de la variable ordinale :")
    print(df_stats.to_string(index=False, formatters={
        "Fréquence": "{:.2%}".format, 
        "Fréquence cumulée": "{:.2%}".format
    }))

    # Affichage des statistiques complémentaires
    print("\n Statistiques complémentaires :")
    print(f"- Mode(s) : {', '.join(map(str, mode_values))}")
    print(f"- Médiane : {mediane}")

    # Affichage des graphiques si demandé
    if include_graph:
        fig, ax = plt.subplots(1, 3, figsize=(15, 5))

        # Diagramme en barres (répartition des valeurs)
        ax[0].bar(df_stats["Valeur"], df_stats["Effectif"], color="skyblue", alpha=0.7)
        ax[0].set_xlabel("Valeurs")
        ax[0].set_ylabel("Effectif")
        ax[0].set_title("Répartition des valeurs ordinales")
        ax[0].tick_params(axis="x", rotation=45)

        # Courbe des effectifs cumulés
        ax[1].plot(df_stats["Valeur"], df_stats["Effectif cumulé"], marker="o", linestyle="-", color="orange")
        ax[1].set_xlabel("Valeurs")
        ax[1].set_ylabel("Effectif cumulé")
        ax[1].set_title("Effectifs cumulés")

        # Boxplot
        if custom_order:
            mapping = {val: i for i, val in enumerate(custom_order)}  # Associer chaque valeur à un nombre
            series_numeric = series.map(mapping).dropna()  # Appliquer la conversion
            ax[2].boxplot(series_numeric, vert=False, patch_artist=True, boxprops=dict(facecolor="lightblue"))
            ax[2].set_title("Boxplot (Valeurs ordinales converties)")

        plt.tight_layout()
        plt.show()



In [ ]:
def statistical_analysis_qualitative_nominal(series: pd.Series, include_graph: bool = False) -> None:
    # Initialiser un compteur pour stocker les fréquences des genres
    compteur_genres = Counter()

    # Parcourir les lignes et compter les occurrences des genres
    for genres in series.dropna():
        liste_genres = [genre.strip() for genre in genres.split(",")]
        compteur_genres.update(liste_genres)

    # Trier les genres par fréquence décroissante
    genres_tries = dict(sorted(compteur_genres.items(), key=lambda x: x[1], reverse=True))

    # Calcul des effectifs, effectifs cumulés, fréquences et fréquences cumulées
    total = sum(genres_tries.values())
    effectifs_cumules = []
    frequences = []
    frequences_cumulees = []

    cumul = 0
    for genre, effectif in genres_tries.items():
        cumul += effectif
        effectifs_cumules.append(cumul)
        frequence = effectif / total
        frequences.append(frequence)
        frequences_cumulees.append(cumul / total)

    # Détermination du mode (les valeurs les plus fréquentes)
    max_effectif = max(genres_tries.values())
    mode_genres = [genre for genre, effectif in genres_tries.items() if effectif == max_effectif]

    # Création du DataFrame pour affichage
    df_stats = pd.DataFrame({
        "Genre": list(genres_tries.keys()),
        "Effectif": list(genres_tries.values()),
        "Effectif cumulé": effectifs_cumules,
        "Fréquence": frequences,
        "Fréquence cumulée": frequences_cumulees
    })

    # Affichage du tableau des statistiques
    print("\nAnalyse des genres de jeux vidéo :")
    print(df_stats.to_string(index=False, formatters={"Fréquence": "{:.2%}".format, "Fréquence cumulée": "{:.2%}".format}))
    
    # Affichage des statistiques supplémentaires
    print("\nMode(s) des genres les plus fréquents :", ", ".join(mode_genres))

    # Affichage des graphiques si demandé
    if include_graph:
        fig, ax = plt.subplots(1, 2, figsize=(12, 5))

        # Histogramme (Bar Chart)
        ax[0].bar(df_stats["Genre"], df_stats["Effectif"], color="skyblue", alpha=0.7)
        ax[0].set_xlabel("Genres")
        ax[0].set_ylabel("Effectif")
        ax[0].set_title("Répartition des genres")
        ax[0].tick_params(axis="x", rotation=45)

        # Diagramme en camembert (Pie Chart)
        ax[1].pie(df_stats["Effectif"], labels=df_stats["Genre"], autopct="%1.1f%%", colors=plt.cm.Paired.colors)
        ax[1].set_title("Répartition des genres (Pie Chart)")

        plt.tight_layout()
        plt.show()

In [ ]:
statistical_analysis_qualitative_nominal(video_games[Variable.genres], include_graph=True)

In [ ]:
statistical_analysis_quantitative(video_games[Variable.overall_review_percentage], cutted=False, include_graph=True)

In [ ]:
statistical_analysis_qualitative_ordinal(video_games[Variable.overall_review], include_graph=True, custom_order=Variable.overall_review_order)

In [ ]:
statistical_analysis_quantitative(video_games[Variable.awards], cutted=False, include_graph=True)

## Problématiques bivariées
- 1. Y a-t-il une relation entre le prix initial (original_price) et le pourcentage de réduction (discount_percentage) des jeux ?
- 2. Le pourcentage de critiques positives (overall_review_%) varie-t-il selon la catégorie du jeu (categories) ?
- 3. Les jeux les mieux notés sont-ils vendus plus chers, même après réduction ?
- 4. Dans le contexte de la distribution numérique de jeux vidéo, les réductions appliquées sur les titres sont un levier marketing majeur pour attirer les consommateurs. Ces remises sont particulièrement visibles lors d’événements promotionnels majeurs comme les soldes Steam, les festivals de jeux indépendants ou les ventes flash. Cependant, une question se pose : les jeux vidéo vendus à un prix plus élevé bénéficient-ils d’un pourcentage de réduction plus important ?
- 5. Comment les caractéristiques d’un jeu vidéo, telles que le prix original, le pourcentage de réduction, le nombre de critiques, les récompenses et la disponibilité de DLC, influencent-elles le pourcentage global des critiques positives (overall_review_%) sur une plateforme comme Steam ? L’objectif est d’identifier les facteurs clés qui contribuent à la réception critique d’un jeu.

## Réponses partielles
- 1. Une analyse préliminaire pourrait consister à calculer le coefficient de corrélation de Pearson entre ces deux variables quantitatives pour évaluer la force et la direction de leur relation (positive, négative ou nulle). On pourrait aussi visualiser cette relation avec un nuage de points (scatter plot).
- 2. On pourrait comparer la moyenne du pourcentage de critiques positives pour chaque catégorie (ex. Solo, Multiplayer) à l’aide d’un test statistique comme une ANOVA ou un test de Kruskal-Wallis si les données ne sont pas normales. Un boxplot pourrait illustrer les différences.
- 3. L’intuition sous-jacente est que la qualité perçue d’un jeu influe sur sa capacité à conserver un prix élevé malgré les remises. Un éditeur peut se permettre de moins baisser le prix d’un jeu plébiscité par les joueurs.
- 4. Cette interrogation soulève une problématique plus large, qui concerne la stratégie commerciale des éditeurs. Ces derniers peuvent, en théorie, appliquer des réductions plus importantes sur les jeux chers afin de compenser un prix de départ dissuasif. À l’inverse, ils pourraient maintenir une réduction faible pour ne pas dévaluer la valeur perçue du produit. 
- 5. Nous supposons que des variables comme un prix original élevé, un pourcentage de réduction important, un grand nombre de critiques, des récompenses et la présence de DLC ont un impact significatif sur le pourcentage de critiques positives. Plus précisément, nous pensons que les jeux avec plus de critiques et de récompenses auront un meilleur pourcentage de critiques positives, tandis qu’un prix élevé pourrait avoir un effet négatif.
## Problématiques gardées
- 3. Les jeux les mieux notés sont-ils vendus plus chers, même après réduction ?
- 4. Dans le contexte de la distribution numérique de jeux vidéo, les réductions appliquées sur les titres sont un levier marketing majeur pour attirer les consommateurs. Ces remises sont particulièrement visibles lors d’événements promotionnels majeurs comme les soldes Steam, les festivals de jeux indépendants ou les ventes flash. Cependant, une question se pose : les jeux vidéo vendus à un prix plus élevé bénéficient-ils d’un pourcentage de réduction plus important ?
- 5. Comment les caractéristiques d’un jeu vidéo, telles que le prix original, le pourcentage de réduction, le nombre de critiques, les récompenses et la disponibilité de DLC, influencent-elles le pourcentage global des critiques positives (overall_review_%) sur une plateforme comme Steam ? L’objectif est d’identifier les facteurs clés qui contribuent à la réception critique d’un jeu.

In [ ]:
# Conversion des prix et pourcentages en valeurs numériques
def clean_price(price_str):
    if pd.isna(price_str) or price_str == 'Free' or price_str == '':
        return 0
    # Extraction du prix numérique (sans devise)
    return float(''.join(char for char in price_str if char.isdigit() or char == '.'))

In [ ]:
def clean_discount(discount_str):
    if pd.isna(discount_str) or discount_str == '':
        return 0
    # Extraction du pourcentage (sans le signe %)
    return float(discount_str.strip('%').strip('-'))

In [ ]:
def statistical_analysis_problem_1(original_price_series: pd.Series, discount_percentage_series: pd.Series, include_graph: bool = False) -> None:
        # Chargement des données
    # La première ligne est déjà définie:
    # video_games = pd.read_csv("data/Jeux_Videos.csv", sep=",")

    clean_price_series = original_price_series.apply(clean_price)
    clean_discount_series = discount_percentage_series.apply(clean_discount)
    
    # Création d'un DataFrame temporaire pour l'analyse
    temp_df = pd.DataFrame({
        'original_price_clean': clean_price_series,
        'discount_percentage_clean': clean_discount_series
    })
    
    # Filtrage des jeux payants avec des données valides
    analysis_df = temp_df[temp_df['original_price_clean'] > 0].copy()
    
    print("============ ANALYSE DE LA RELATION PRIX INITIAL - RÉDUCTION ============")
    
    # 1. Statistiques descriptives
    print("\n1. Statistiques descriptives:")
    print(analysis_df.describe())
    
    # Vérification si les données sont suffisantes pour l'analyse
    if len(analysis_df) < 2:
        print("Pas assez de données pour effectuer l'analyse de corrélation.")
        return
    
    # 2. Analyse de corrélation
    correlation = analysis_df['original_price_clean'].corr(analysis_df['discount_percentage_clean'])
    print(f"\n2. Coefficient de corrélation entre prix initial et réduction: {correlation:.4f}")
    
    # Test de signification statistique (p-value)
    corr_test = stats.pearsonr(analysis_df['original_price_clean'], analysis_df['discount_percentage_clean'])
    print(f"   p-value: {corr_test[1]:.4f}")
    
    # Interprétation du coefficient de corrélation
    print("\n   Interprétation du coefficient de corrélation:")
    if abs(correlation) < 0.1:
        strength = "très faible voire inexistante"
    elif abs(correlation) < 0.3:
        strength = "faible"
    elif abs(correlation) < 0.5:
        strength = "modérée"
    elif abs(correlation) < 0.7:
        strength = "forte"
    else:
        strength = "très forte"
        
    direction = "positive" if correlation > 0 else "négative"
    significance = "statistiquement significative" if corr_test[1] < 0.05 else "non statistiquement significative"
    
    print(f"   La corrélation est {strength} et {direction} ({significance}).")
    
    # 3. Modélisation linéaire
    X = analysis_df[['original_price_clean']]
    y = analysis_df['discount_percentage_clean']
    
    model = LinearRegression()
    model.fit(X, y)
    
    print(f"\n3. Modèle linéaire:")
    print(f"   Équation: discount_percentage = {model.coef_[0]:.4f} × original_price + {model.intercept_:.4f}")
    print(f"   R² (coefficient de détermination): {model.score(X, y):.4f}")
    
    # Interprétation du modèle
    print("\n   Interprétation du modèle:")
    if model.coef_[0] > 0:
        print(f"   Pour chaque augmentation de 1 unité du prix initial, le pourcentage de réduction augmente en moyenne de {model.coef_[0]:.4f} points.")
    else:
        print(f"   Pour chaque augmentation de 1 unité du prix initial, le pourcentage de réduction diminue en moyenne de {abs(model.coef_[0]):.4f} points.")
    
    # 4. Analyse par tranches de prix
    analysis_df['price_range'] = pd.cut(analysis_df['original_price_clean'], 
                                        bins=[0, 10, 20, 30, 40, 50, float('inf')],
                                        labels=['0-10', '10-20', '20-30', '30-40', '40-50', '50+'])
    
    price_range_analysis = analysis_df.groupby('price_range')['discount_percentage_clean'].agg(['mean', 'count'])
    print("\n4. Réduction moyenne par tranche de prix:")
    print(price_range_analysis)
    
    # 5. Visualisations (conditionnelles)
    if include_graph:
        # Nuage de points avec ligne de tendance
        plt.figure(figsize=(10, 6))
        sns.scatterplot(x='original_price_clean', y='discount_percentage_clean', data=analysis_df, alpha=0.6)
        sns.regplot(x='original_price_clean', y='discount_percentage_clean', data=analysis_df, 
                    scatter=False, color='red', line_kws={"linestyle": "--"})
        plt.title('Relation entre prix initial et pourcentage de réduction')
        plt.xlabel('Prix initial')
        plt.ylabel('Pourcentage de réduction')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # Graphique à barres par tranche de prix
        plt.figure(figsize=(10, 6))
        sns.barplot(x=price_range_analysis.index, y='mean', data=price_range_analysis.reset_index())
        plt.title('Pourcentage de réduction moyen par tranche de prix')
        plt.xlabel('Tranche de prix')
        plt.ylabel('Réduction moyenne (%)')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.show()
    
        # 6. Conclusion
        print("\n5. Conclusion:")
        if abs(correlation) < 0.1:
            print("   Il n'existe pas de relation linéaire claire entre le prix initial et le pourcentage de réduction.")
        else:
            print(f"   Il existe une relation {strength} {direction} entre le prix initial et le pourcentage de réduction.")
            
        if model.score(X, y) < 0.3: 
            print("   Le modèle linéaire explique très peu la variance des réductions, suggérant que d'autres facteurs influencent davantage les politiques de réduction.")
        
        print("\n============================================================")


In [ ]:
statistical_analysis_problem_1(Variable.original_price, Variable.discount_percentage, include_graph = True)